In [7]:
import requests
import json
import numpy as np

In [11]:
url = "https://shopee.vn/api/v4/official_shop/get_shops?category_id=11035567"

headers = {
 'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36 Edg/115.0.0.0'
}
response = requests.get(url, headers= headers).json()

data = np.array(response)


In [ ]:
for i in data['data']['official_shops']:
    row = i['shopid'], i['shop_name']
    print(row)

In [13]:
print(data)

{'error': 0, 'error_msg': None, 'data': {'total': 218, 'official_shops': [{'userid': 40343949, 'username': 'sp.btw2', 'shopid': 40342563, 'shop_name': 'Aviano Menswear', 'logo': '3d150be843d6d2ef804953b0d481d2da', 'logo_pc': '513f589c10254512669d3d8b50a6dea7', 'shop_collection_id': 0, 'ctime': 1620981895, 'brand_label': 0}, {'userid': 225913958, 'username': 'poloman.vn', 'shopid': 225909574, 'shop_name': 'HAPPYHOW', 'logo': '123cc164a4752e490fabf1615d426e24', 'logo_pc': 'd1aa441638492fb588741d01af604811', 'shop_collection_id': 0, 'ctime': 1638428571, 'brand_label': 0}, {'userid': 68615221, 'username': 'tsla.store', 'shopid': 68613764, 'shop_name': 'TSLA Store Quần áo legging nam', 'logo': '43379c63d96678378ac4e96b9143ad76', 'logo_pc': '5c1936f863ed34b0e77299b2dbb96365', 'shop_collection_id': 0, 'ctime': 1650257648, 'brand_label': 0}, {'userid': 24711470, 'username': 'coolmate.vn', 'shopid': 24710134, 'shop_name': '\x08Coolmate - Official Store', 'logo': 'f1e3bd8688dd9a60d16b10bfb3ec846